In [5]:
# 자동 미분 및 그라디언트
# 자동 미분은 신경망 학습 위한 역전 파와 같은 기계 학습 알고리즘을 구현하는 데 유용

# 컴퓨팅 그리디언트
# 자동으로 미분하기 위해 Tensorflow는 전달 과정에서 어떤 작업이 어떤 순서로 발생하는지 기억해야 한다
# 그런 다음 역방향 패스 동안 Tensorflow는 연산 목록을 역순으로 이동하여 그라디언트를 계산한다

# GradientTape
# Tensorflow는 자동 차별화를 위한 tf.GradientTape API를 제공한다
# 일부 입력, 일반적으로 tf.Variable과 관련하여 계산 기울기를 계산한다
# Tensorflow는 tf.GradientTape 의 context 내에서 실행 된 관련 작업을 'Tape'에 기록한다
# 그런 다음, Tensorflow는 해당 테이프를 사용하여 '역 모드 차별화'를 사용하여 '기록 된' 계산의 기울기를 계산한다

# 간단한 예로 살펴보자

In [6]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

In [7]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x**2 # x라는 값을 넣어서 제곱하여 y값 구하는 게 forward pass, 순방향 연산

In [8]:
# tf.Variable 이라는 변수가 들어오면 이것을 추적한다
# Variable이 어떻게 loss값, target 값에 영향을 주는지
# x제곱 미분값 : 2x

In [9]:
# 일부 작업을 기록한 후에는 GradientTape.gradient(target, sources)를 사용하여 
# 일부 소스(종종 모델 변수)에 상대적인 일부 대상(종종 손실)의 기울기를 계산한다
# dy = 2x * dx
dy_dx = tape.gradient(y, x) # y값으로 x값의 기울기를 추적하는 것이 backward pass, 미분 값을 구하는 것이다
dy_dx.numpy()

6.0

In [10]:
# x **2 -> 2x

In [11]:
# 자동미분은 순방향으로 가는 forward pass를 거꾸로 갔을 때 미분 값을 구하는 것이다

In [13]:
w1 = tf.Variable(tf.random.normal((3, 2)), name = 'w1')
b1 = tf.Variable(tf.zeros(2, dtype=tf.float32), name = 'b1')
x1 = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
    y1 = x1 @ w1 + b1
    loss = tf.reduce_mean(y1**2)

In [16]:
# 두 변수 모두에 대해 y의 기울기를 얻으려면 둘 다 소스로 gradient 방법을 전달할 수 있다
# 테이프는 소스가 전달되는 방식에 대해 융통성이 있으며 중첩 된 목록 또는 사전 조합을 허용하고
# 동일한 방식으로 구조화 된 gradient를 return 한다

[dl_dw1, dl_db1] = tape.gradient(loss, [w1, b1]) # loss값에 w1, b1이 어떻게 영향을 미치는지 미분값을 구해서 알 수 있다

dl_dw1, dl_db1

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 1.1216218, -1.8027582],
        [ 2.2432437, -3.6055164],
        [ 3.3648655, -5.4082747]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 1.1216218, -1.8027582], dtype=float32)>)

In [17]:
print(w1.shape)
print(dl_dw1.shape)

(3, 2)
(3, 2)


In [26]:
# 변수를 사전 전달하여 그라이어트 계산

my_vars = {
    'w2' : tf.Variable(tf.random.normal((3, 2)), name = 'w2'),
    'b2' : tf.Variable(tf.zeros(2, dtype = tf.float32), name = 'b2')
}


x2 = [[1., 2., 3.]]

with tf.GradientTape() as tape:
    y2 = x2 @ my_vars['w2'] + my_vars['b2']
    loss2 = tf.reduce_mean(y2**2)

grads = tape.gradient(loss2, my_vars)

In [27]:
grads

{'w2': <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ -4.942117 ,  -2.4153812],
        [ -9.884234 ,  -4.8307624],
        [-14.826351 ,  -7.2461433]], dtype=float32)>,
 'b2': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-4.942117 , -2.4153812], dtype=float32)>}

In [ ]:
# 모델에 대한 그라디언트

layer = tf.keras.layers.Dense(2, activation='relu')
x3 = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape :
    # Froward pass
    y3 = layer(x3)
    loss3 = tf.reduce_mean(y**2)
    
    # calculate gradients with repect to every trainable variabel
    grad = tape.gradient(loss, layer.trainable_variables)